In [1]:
#aysenur

In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl
from pytorch_lightning.metrics import F1
from pytorch_lightning.loggers import WandbLogger

from pytorch_lightning.loggers import TensorBoardLogger

In [4]:
from DataPreparation import get_data

In [5]:
CURRENCY_LST = ['BTC', 'ETH']#, 'LTC']
FREQUENCY = "D"

WINDOW_SIZE = 14
NEUTRAL_QUANTILE = 0.25 # çok olduğunu düşünüyorum
N_CLASSES = 2

In [6]:
TRAIN_PERCENTAGE, VAL_PERCENTAGE, TEST_PERCENTAGE = 0.90, 0.007, 0.023

In [7]:
LOSS_WEIGHT_CALCULATE = False

In [8]:
BATCH_SIZE= 16

In [9]:
X, y, features, dfs = get_data(CURRENCY_LST,
                                N_CLASSES,
                                 FREQUENCY, 
                                 WINDOW_SIZE,
                                 neutral_quantile = NEUTRAL_QUANTILE,
                                 log_price=True,
                                 remove_trend=True,
                                 include_indicators = False,
                                 include_imfs = False
                                )

In [10]:
N_CURRENCIES = X.shape[0]
INPUT_FEATURE_SIZE = X.shape[-1]

N_CURRENCIES, INPUT_FEATURE_SIZE

(2, 1)

In [11]:
class MultiTimeSeriesDataset(Dataset):
    def __init__(self, 
                 n_currencies,
                 x: np.ndarray, 
                 y: np.ndarray,
                 data_use_type,
                 train_percentage = TRAIN_PERCENTAGE,
                 val_percentage = VAL_PERCENTAGE,
                 test_percentage = TEST_PERCENTAGE,
                 seq_len = WINDOW_SIZE, 
                 ):
        
        self.x = torch.tensor(x[:n_currencies]).float()
        self.y = torch.tensor(y[:n_currencies]).long()
        self.seq_len = seq_len
        self.data_use_type = data_use_type
        
        #self.train_size = int(len(self.x[0]) * train_percentage)
        self.val_size = int(len(self.x[0]) * val_percentage)
        self.test_size = int(len(self.x[0]) * test_percentage)
        self.train_size = len(self.x[0]) - self.val_size - self.test_size 
        
        self.train_mean = [self.x[i][:self.train_size].mean() for i in range(n_currencies)]
        self.train_std = [self.x[i][:self.train_size].std() for i in range(n_currencies)]
        
        self.train_min = [self.x[i][:self.train_size].min() for i in range(n_currencies)]
        self.train_max = [self.x[i][:self.train_size].max() for i in range(n_currencies)]
        
    def __len__(self):
        
        if self.data_use_type == "train":
            return self.train_size - ( self.seq_len)

        elif self.data_use_type == "val":
            return self.val_size
  
        else:
            return self.test_size
        
    
    def __getitem__(self, index):
        
        item = dict()
        
        if self.data_use_type =="val":
            index = self.train_size + index - self.seq_len
            
        elif self.data_use_type =="test":
            index = self.train_size + self.val_size + index - self.seq_len
        
        for i in range(N_CURRENCIES):
            window = self.x[i][index:index+self.seq_len]
            window = (window -self.train_mean[i]) / self.train_std[i]
            
            item["currency_" + str(i) + "_window"] = window
            item["currency_" + str(i) + "_label"]  = self.y[i][index+self.seq_len]

        return item

In [12]:
train_dataset, val_dataset, test_dataset = [MultiTimeSeriesDataset(N_CURRENCIES, X, y, dtype) for dtype in ['train', 'val', 'test']]

In [13]:
len(train_dataset), len(val_dataset), len(test_dataset)

(1299, 9, 31)

In [14]:
if LOSS_WEIGHT_CALCULATE:
    loss_weights = []
    for n in range(N_CURRENCIES):
        train_labels = [int(train_dataset[i]["currency_"+ str(n)+"_label"] )for i in range(train_dataset.__len__())]
        samples_size = pd.DataFrame({"label": train_labels}).groupby("label").size().to_numpy()
        print(samples_size)
        loss_weights.append((1 / samples_size) * sum(samples_size)/2)
    loss_weights
else:
    loss_weights = None

loss_weights

In [15]:
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    attn_logits = torch.matmul(q, k.transpose(-2, -1))
    attn_logits = attn_logits / math.sqrt(d_k)
    if mask is not None:
        attn_logits = attn_logits.masked_fill(mask == 0, -9e15)
    attention = F.softmax(attn_logits, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

In [16]:
class MultiheadAttention(nn.Module):
    
    def __init__(self, input_dim, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0, "Embedding dimension must be 0 modulo number of heads."
        
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        # Stack all weight matrices 1...h together for efficiency
        # Note that in many implementations you see "bias=False" which is optional
        self.qkv_proj = nn.Linear(input_dim, 3*embed_dim)
        self.o_proj = nn.Linear(embed_dim, embed_dim)
        
        self._reset_parameters()
        
        
    def _reset_parameters(self):
        # Original Transformer initialization, see PyTorch documentation
        nn.init.xavier_uniform_(self.qkv_proj.weight)
        self.qkv_proj.bias.data.fill_(0)
        nn.init.xavier_uniform_(self.o_proj.weight)
        self.o_proj.bias.data.fill_(0)
        
        
    def forward(self, x, mask=None, return_attention=False):
        batch_size, seq_length, embed_dim = x.size()
        qkv = self.qkv_proj(x)
        
        # Separate Q, K, V from linear output
        qkv = qkv.reshape(batch_size, seq_length, self.num_heads, 3*self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3) # [Batch, Head, SeqLen, Dims]
        q, k, v = qkv.chunk(3, dim=-1)
        
        # Determine value outputs
        values, attention = scaled_dot_product(q, k, v, mask=mask)
        values = values.permute(0, 2, 1, 3) # [Batch, SeqLen, Head, Dims]
        values = values.reshape(batch_size, seq_length, embed_dim)
        o = self.o_proj(values)
        
        if return_attention:
            return o, attention
        else:
            return o

In [17]:
class EncoderBlock(nn.Module):
    
    def __init__(self, input_dim, num_heads, dim_feedforward, dropout=0.0):
        """
        Inputs:
            input_dim - Dimensionality of the input
            num_heads - Number of heads to use in the attention block
            dim_feedforward - Dimensionality of the hidden layer in the MLP
            dropout - Dropout probability to use in the dropout layers
        """
        super().__init__()
        
        # Attention layer
        self.self_attn = MultiheadAttention(input_dim, input_dim, num_heads)
        
        # Two-layer MLP
        self.linear_net = nn.Sequential(
            nn.Linear(input_dim, dim_feedforward),
            nn.Dropout(dropout),
            nn.ReLU(inplace=True),
            nn.Linear(dim_feedforward, input_dim)
        )
        
        # Layers to apply in between the main layers
        self.norm1 = nn.LayerNorm(input_dim)
        self.norm2 = nn.LayerNorm(input_dim)
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, x, mask=None):
        # Attention part
        attn_out = self.self_attn(x, mask=mask)
        x = x + self.dropout(attn_out)
        x = self.norm1(x)
        
        # MLP part
        linear_out = self.linear_net(x)
        x = x + self.dropout(linear_out)
        x = self.norm2(x)
        
        return x

In [18]:
class TransformerEncoder(nn.Module):
    
    def __init__(self, num_layers, **block_args):
        super().__init__()
        self.layers = nn.ModuleList([EncoderBlock(**block_args) for _ in range(num_layers)])
        
    
    def forward(self, x, mask=None):
        for l in self.layers:
            x = l(x, mask=mask)
        return x
    
    
    def get_attention_maps(self, x, mask=None):
        attention_maps = []
        for l in self.layers:
            _, attn_map = l.self_attn(x, mask=mask, return_attention=True)
            attention_maps.append(attn_map)
            x = l(x)
        return attention_maps

In [19]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        """
        Inputs
            d_model - Hidden dimensionality of the input.
            max_len - Maximum length of a sequence to expect.
        """
        super().__init__()

        # Create matrix of [SeqLen, HiddenDim] representing the positional encoding for max_len inputs
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        
        # register_buffer => Tensor which is not a parameter, but should be part of the modules state.
        # Used for tensors that need to be on the same device as the module.
        # persistent=False tells PyTorch to not add the buffer to the state dict (e.g. when we save the model) 
        self.register_buffer('pe', pe, persistent=False)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x

In [20]:
NUM_TASKS =2

In [21]:
class TransformerPredictor(pl.LightningModule):
    
    
    def __init__(self, input_dim, model_dim, num_classes, num_heads, num_layers, lr, warmup, max_iters, dropout=0.0, input_dropout=0.0):
        """
        Inputs:
            input_dim - Hidden dimensionality of the input
            model_dim - Hidden dimensionality to use inside the Transformer
            num_classes - Number of classes to predict per sequence element
            num_heads - Number of heads to use in the Multi-Head Attention blocks
            num_layers - Number of encoder blocks to use.
            lr - Learning rate in the optimizer
            warmup - Number of warmup steps. Usually between 50 and 500
            max_iters - Number of maximum iterations the model is trained for. This is needed for the CosineWarmup scheduler
            dropout - Dropout to apply inside the model
            input_dropout - Dropout to apply on the input features
        """
        super().__init__()
        self.save_hyperparameters()
        self._create_model()
        
    
    def _create_model(self):
        # Input dim -> Model dim
        self.input_net = nn.Linear(self.hparams.input_dim, self.hparams.model_dim)
        
        # Positional encoding for sequences
        self.positional_encoding = PositionalEncoding(d_model=self.hparams.model_dim)
        # Transformer
        self.transformer = TransformerEncoder(num_layers=self.hparams.num_layers,
                                              input_dim=self.hparams.model_dim,
                                              dim_feedforward=2*self.hparams.model_dim,
                                              num_heads=self.hparams.num_heads,
                                              dropout=self.hparams.dropout)
        # Output classifier per sequence lement
        self.output_net = [nn.Sequential(
            nn.Linear(self.hparams.model_dim, self.hparams.model_dim),
            nn.LayerNorm(self.hparams.model_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(self.hparams.dropout),
            nn.Linear(self.hparams.model_dim, self.hparams.num_classes)
        ) ]* NUM_TASKS
        
        self.output_net = torch.nn.ModuleList(self.output_net)
        
    
    def forward(self, x, i, mask=None, add_positional_encoding=True):
        """
        Inputs:
            x - Input features of shape [Batch, SeqLen, input_dim]
            mask - Mask to apply on the attention outputs (optional)
            add_positional_encoding - If True, we add the positional encoding to the input.
                                      Might not be desired for some tasks.
        """
        x = self.input_net(x)
        if add_positional_encoding:
            x = self.positional_encoding(x)
        x = self.transformer(x, mask=mask)
        x = self.output_net[i](x)
        return x
    
    
    @torch.no_grad()
    def get_attention_maps(self, x, mask=None, add_positional_encoding=True):
        """
        Function for extracting the attention matrices of the whole Transformer for a single batch.
        Input arguments same as the forward pass.
        """
        x = self.input_net(x)
        if add_positional_encoding:
            x = self.positional_encoding(x)
        attention_maps = self.transformer.get_attention_maps(x, mask=mask)
        return attention_maps
    
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
        
        # We don't return the lr scheduler because we need to apply it per iteration, not per epoch
#         self.lr_scheduler = CosineWarmupScheduler(optimizer, 
#                                                   warmup=self.hparams.warmup, 
#                                                   max_iters=self.hparams.max_iters)
        return optimizer
    
    
#     def optimizer_step(self, *args, **kwargs):
#         super().optimizer_step(*args, **kwargs)
#         self.lr_scheduler.step() # Step per iteration
    
    
    def training_step(self, batch, batch_idx):
        raise NotImplementedError
    
    
    def validation_step(self, batch, batch_idx):
        raise NotImplementedError    
    
    
    def test_step(self, batch, batch_idx):
        raise NotImplementedError   

In [22]:
class TradePredictor(TransformerPredictor):
    
    def _calculate_loss(self, batch, mode="train"):
        f1_score = pl.metrics.F1(num_classes=N_CLASSES, average="macro")
        
        loss = (torch.tensor(0.0, device="cuda:0", requires_grad=True) + \
                torch.tensor(0.0, device="cuda:0", requires_grad=True)) 
        for i in range(2):
            x, y = batch["currency_" + str(i) + "_window"], batch["currency_" + str(i) + "_label"]
  
            preds = self.forward(x, i, add_positional_encoding=True) # No positional encodings as it is a set, not a sequence!
            preds = preds.squeeze(dim=-1) # Shape: [Batch_size, set_size]
            loss += F.cross_entropy(preds, y) # Softmax/CE over set dimension
            
            acc = (preds.argmax(dim=-1) == y).float().mean()
            #f1 = f1_score(torch.max(preds, dim=1)[1], y)
            
            self.log("currency_" + str(i) + "%s_loss" % mode, loss)
            self.log("currency_" + str(i) + "%s_acc" % mode, acc, on_step=False, on_epoch=True)
            #self.log("currency_" + str(i) + "%s_f1" % mode, acc, on_step=False, on_epoch=True)
        
        loss = loss / torch.tensor(2)
        return loss
        
    def training_step(self, batch, batch_idx):
        loss = self._calculate_loss(batch, mode="train")
        return loss
    
    def validation_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="val")
    
    def test_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="test")

In [31]:
trainer = pl.Trainer(#default_root_dir=root_dir, 
                     #checkpoint_callback=ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc"),
                     gpus=-1 , 
                     max_epochs=100,
                     gradient_clip_val=2,
                     progress_bar_refresh_rate=1)

trainer.logger._default_hp_metric = TensorBoardLogger("../output/models/lstm_model_logs", name="multi_head") # Optional logging argument that we don't need


model = TradePredictor(max_iters=trainer.max_epochs*len(train_dataset), 
                       input_dim=1,
                      model_dim=32,
                      num_heads=4,
                      num_classes=1,
                      num_layers=4,
                      dropout=0.1,
                      input_dropout=0.1,
                      lr=5e-4,
                      warmup=50)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [32]:
trainer.fit(model, 
            DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle = True), 
            DataLoader(val_dataset, batch_size=BATCH_SIZE))

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type               | Params
-----------------------------------------------------------
0 | input_net           | Linear             | 64    
1 | positional_encoding | PositionalEncoding | 0     
2 | transformer         | TransformerEncoder | 34.2 K
3 | output_net          | ModuleList         | 1.2 K 
-----------------------------------------------------------
35.4 K    Trainable params
0         Non-trainable params
35.4 K    Total params
0.142     Total estimated model params size (MB)


1

In [25]:
trainer.test(model, test_dataloaders=DataLoader(test_dataset, batch_size=32), verbose=True)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'currency_0test_acc': 0.7096773982048035,
 'currency_0test_loss': 1.129563808441162,
 'currency_1test_acc': 0.7096773982048035,
 'currency_1test_loss': 1.129563808441162}
--------------------------------------------------------------------------------


[{'currency_0test_loss': 1.129563808441162,
  'currency_0test_acc': 0.7096773982048035,
  'currency_1test_loss': 1.129563808441162,
  'currency_1test_acc': 0.7096773982048035}]

In [33]:
for n in range(N_CURRENCIES):
    labels = [int(test_dataset[i]["currency_"+ str(n)+"_label"] )for i in range(test_dataset.__len__())]
    print(pd.DataFrame({"label": labels}).groupby("label").size())

label
0    18
1    13
dtype: int64
label
0    15
1    16
dtype: int64


In [27]:
#logger = WandbLogger(name='lstm.v1',project='pytorchlightning')
logger = TensorBoardLogger("../output/models/lstm_model_logs", name="self_attention")

In [28]:
model = Attention_based_classification_model()
trainer = pl.Trainer(gpus=-1, 
                     max_epochs= 150,
                     logger = logger)

NameError: name 'TransformerEncoderLayer' is not defined

In [ ]:
trainer.fit(model)

In [ ]:
trainer.test()